In [3]:
import time
import pandas as pd
import os

# Start the timer
start_time = time.time()

#input file
file_path = "../Monthly_Input_Files/Medi-Cal Rates 07.15.2024.xlsx"
date_part = file_path.split(" ")[-1].split(".")  
split_date = '-'.join(date_part[0:3])
month, day, year = split_date.split('-')
if len(year) == 2:
    year = '20' + year
date = f'{year}-{month}-01'

               
df = pd.read_excel(file_path, skiprows=1) 
# Define new column names based on the data structure in your file
df.columns = ['Proc_Type', 'Proc_Code', 'Procedure_Description', 'Unit_Value',
              'Basic_Rate', 'Child_Rate', 'ER_Rate', 'Conv_Ind', 'ER_Ind',
              'Cutback_Ind', 'Prof_%', 'Rental_Rate', 'Non_Physn_Med_Prac_Ind']

 
# Define columns for Augment Code based on the data structures
#df_ac.columns = ['Proc_Code','Procedure_Description_for_Augmentation', 'Basic_rate', 'Proc_Type']

# Remove the $ sign and convert the column to numeric, coerce errors to NaN
df['Basic_Rate'] = pd.to_numeric(df['Basic_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
df['Child_Rate'] = pd.to_numeric(df['Child_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
df['ER_Rate'] = pd.to_numeric(df['ER_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
df['Rental_Rate'] = pd.to_numeric(df['Rental_Rate'].str.replace('[\$,]', '', regex=True), errors='coerce').round(2)
df['Prof_%'] = pd.to_numeric(df['Prof_%'], errors='coerce').round(2)

def Purchase_Calculation(df,proc_type):
    filtered_df_1 = df.copy()
    filtered_df_1 = filtered_df_1[filtered_df_1['Basic_Rate'] > 0.01]
    filtered_df_1['FEE'] = filtered_df_1['Basic_Rate'].round(2)
    filtered_df_1['MODIFIER_1']='NU'
    filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
    filtered_df_1['AGE_UNIT']=''
    filtered_df_1['FROM_AGE']=''
    filtered_df_1['THROUGH_AGE']=''
    filtered_df_1['PLACE_OF_SERVICE_CODE']=''
    
    final_df_a = filtered_df_1[['SVC_CD', 'MODIFIER_1', 'FEE', 'AGE_UNIT', 'FROM_AGE', 'THROUGH_AGE', 'PLACE_OF_SERVICE_CODE']]
    return final_df_a

def Rental_Calculation(df,proc_type):
    filtered_df_1 = df.copy()

    filtered_df_1['FEE']=filtered_df_1['Rental_Rate'].round(2)
    filtered_df_1['MODIFIER_1']='RR'
    filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
    filtered_df_1['AGE_UNIT']=''
    filtered_df_1['FROM_AGE']=''
    filtered_df_1['THROUGH_AGE']=''
    filtered_df_1['PLACE_OF_SERVICE_CODE']=''
    final_df_a = filtered_df_1[['SVC_CD', 'MODIFIER_1', 'FEE', 'AGE_UNIT', 'FROM_AGE', 'THROUGH_AGE', 'PLACE_OF_SERVICE_CODE']]
    return final_df_a

def Basic_Rate_Calculation(df,proc_type):
    filtered_df_1 = df[(df['Basic_Rate'] > 0)].copy()
    filtered_df_2 = df[(df['Basic_Rate'] > 0) & (df['Prof_%'] > 0)].copy()
    filtered_df_3 = df[(df['Basic_Rate'] > 0) & (df['Prof_%'] !=0) & (1- df['Prof_%'] > 0)].copy()
    
    filtered_df_1['FEE']=filtered_df_1['Basic_Rate'].round(2)
    filtered_df_1['MODIFIER_1']=''
    filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
    # Create a professional rate DataFrame
    prof_df = filtered_df_2.copy()
    prof_df['FEE'] = (prof_df['Basic_Rate'] * prof_df['Prof_%']).round(2)
    prof_df['MODIFIER_1'] = '26'
    prof_df['SVC_CD']=prof_df['Proc_Code']
 
    # Create a technical rate DataFrame
    tech_df = filtered_df_3.copy()
    tech_df['FEE'] = (tech_df['Basic_Rate'] * (1 - tech_df['Prof_%'])).round(2)
    tech_df['MODIFIER_1'] = 'TC'
    tech_df['SVC_CD']=tech_df['Proc_Code']
 
    # Append the technical DataFrame to the professional DataFrame
    result_df = pd.concat([filtered_df_1,prof_df, tech_df], ignore_index=True)
    result_df['AGE_UNIT']=''
    result_df['FROM_AGE']=''
    result_df['THROUGH_AGE']=''
    result_df['PLACE_OF_SERVICE_CODE']=''
    final_df_a = result_df[['SVC_CD', 'MODIFIER_1', 'FEE', 'AGE_UNIT', 'FROM_AGE', 'THROUGH_AGE', 'PLACE_OF_SERVICE_CODE']]
    
    return final_df_a

def Child_Rate_Calculation(df,proc_type):
    # Filter DataFrame for rows where both Prof_% and Basic_Rate are greater than 0
    filtered_df_1 = df[(df['Proc_Type']==proc_type) & (df['Child_Rate'] > 0)].copy()
    filtered_df_2 = df[(df['Proc_Type']==proc_type) & (df['Child_Rate'] > 0) & (df['Prof_%'] > 0)].copy()
    filtered_df_3 = df[(df['Proc_Type']==proc_type) & (df['Child_Rate'] > 0) & (df['Prof_%'] !=0) & (1- df['Prof_%'] > 0)].copy()
    result_df=pd.DataFrame()
    if len(filtered_df_1)>0:
        filtered_df_1['FEE']=filtered_df_1['Child_Rate'].round(2)
        filtered_df_1['MODIFIER_1']=''
        filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
    # Create a professional rate DataFrame
        prof_df = filtered_df_2.copy()
        prof_df['FEE'] = (prof_df['Child_Rate'] * prof_df['Prof_%']).round(2)
        prof_df['MODIFIER_1'] = '26'
        prof_df['SVC_CD']=prof_df['Proc_Code']
 
    # Create a technical rate DataFrame
        tech_df = filtered_df_3.copy()
        tech_df['FEE'] = (tech_df['Child_Rate'] * (1 - tech_df['Prof_%'])).round(2)
        tech_df['MODIFIER_1'] = 'TC'
        tech_df['SVC_CD']=tech_df['Proc_Code']
 
    # Append the technical DataFrame to the professional DataFrame
        result_df = pd.concat([filtered_df_1,prof_df, tech_df], ignore_index=True)
        result_df['AGE_UNIT']='3'
        result_df['FROM_AGE']='0'
        result_df['THROUGH_AGE']='18'
        result_df['PLACE_OF_SERVICE_CODE']=''
        result_df = result_df[['SVC_CD', 'MODIFIER_1', 'FEE', 'AGE_UNIT', 'FROM_AGE', 'THROUGH_AGE', 'PLACE_OF_SERVICE_CODE']]
    return result_df

def ER_Rate_Calculation(df,proc_type):
    # Filter DataFrame for rows where both Prof_% and Basic_Rate are greater than 0
    filtered_df_1 = df[(df['Proc_Type']==proc_type) & (df['ER_Rate'] > 0) & (df['ER_Ind']>0)].copy()
    filtered_df_2 = df[(df['Proc_Type']==proc_type) & (df['ER_Rate'] > 0) & (df['Prof_%'] > 0) & (df['ER_Ind']>0)].copy()
    filtered_df_3 = df[(df['Proc_Type']==proc_type) & (df['ER_Rate'] > 0) & (df['Prof_%'] !=0) & (1- df['Prof_%'] > 0) & (df['ER_Ind']>0)].copy()
    
    filtered_df_1['FEE']=filtered_df_1['ER_Rate'].round(2)
    filtered_df_1['MODIFIER_1']=''
    filtered_df_1['SVC_CD']=filtered_df_1['Proc_Code']
    # Create a professional rate DataFrame
    prof_df = filtered_df_2.copy()
    prof_df['FEE'] = (prof_df['ER_Rate'] * prof_df['Prof_%']).round(2)
    prof_df['MODIFIER_1'] = '26'
    prof_df['SVC_CD']=prof_df['Proc_Code']
 
    # Create a technical rate DataFrame
    tech_df = filtered_df_3.copy()
    tech_df['FEE'] = (tech_df['ER_Rate'] * (1 - tech_df['Prof_%'])).round(2)
    tech_df['MODIFIER_1'] = 'TC'
    tech_df['SVC_CD']=tech_df['Proc_Code']
 
    # Append the technical DataFrame to the professional DataFrame
    result_df = pd.concat([filtered_df_1,prof_df, tech_df], ignore_index=True)
    result_df['AGE_UNIT']=''
    result_df['FROM_AGE']=''
    result_df['THROUGH_AGE']=''
    result_df['PLACE_OF_SERVICE_CODE']='23'
    return result_df[['SVC_CD', 'MODIFIER_1', 'FEE', 'AGE_UNIT', 'FROM_AGE', 'THROUGH_AGE', 'PLACE_OF_SERVICE_CODE']]

def anes_calculation(df,proc_type):
    # Filter DataFrame for rows where both Prof_% and Basic_Rate are greater than 0
    filtered_df_1 = df.copy()
    
    filtered_df_1['CPT Code']=filtered_df_1['Proc_Code']
    filtered_df_1['CPT Descriptor']=filtered_df_1['Procedure_Description']
    filtered_df_1['Base Unit Value']=filtered_df_1['Unit_Value']
    
    filtered_df_1['AGE_UNIT']=''
    filtered_df_1['FROM_AGE']=''
    filtered_df_1['THROUGH_AGE']=''
    filtered_df_1['PLACE_OF_SERVICE_CODE']=''
    final_df_a = filtered_df_1[['CPT Code', 'CPT Descriptor', 'Base Unit Value']]
    
    return final_df_a
# Create a dictionary to hold dataframes for each a_type
df_dict = {}

# Get unique Proc_Types
Proc_Types = df['Proc_Type'].unique()

# Create a dataframe for each Proc_Types and store it in the dictionary
for Proc_Type in Proc_Types:
    df_dict[Proc_Type] = df[df['Proc_Type'] == Proc_Type]

# Initialize a new dictionary to hold the filtered dataframes
filtered_df_dict = {}



In [7]:
if '1' in df_dict:
    filtered_df_dict_1 = df_dict['1'][(~df_dict['1']['Proc_Code'].str.startswith('A0')) & (df_dict['1']['Rental_Rate'] == 0)].copy()
    filtered_df_dict_1 = filtered_df_dict_1[(filtered_df_dict_1['Basic_Rate'] > 0)]
    Basic_df=Basic_Rate_Calculation(filtered_df_dict_1,'1')
    Child_df=Child_Rate_Calculation(filtered_df_dict_1,'1')
    ER_df =ER_Rate_Calculation(filtered_df_dict_1,'1')
    final_df = pd.concat([Basic_df,Child_df,ER_df], ignore_index=True)
    final_df.to_csv(f'Mcal_Allied.{date}.txt', sep='|', index=False)
    #final_df.to_csv(f'Mcal_Allied.{date}.txt', sep='|', index=False)

    filtered_df_dict_2 = df_dict['1'][(~df_dict['1']['Proc_Code'].str.startswith('A0')) & (df_dict['1']['Rental_Rate'] > 0)].copy()
    Purchase_df = Purchase_Calculation(filtered_df_dict_2,'1')
    Rental_df = Rental_Calculation(filtered_df_dict_2,'1')
    final_df_2 = pd.concat([Purchase_df,Rental_df], ignore_index=True)
    final_df_2.to_csv(f'Mcal_DME.{date}.txt', sep='|', index=False)
    #final_df_2.to_csv(f'Mcal_DME.{date}.txt', sep='|', index=False) 
print("code executed succesfully")

code executed succesfully


In [5]:
filtered_df_dict_2


,Proc_Type,Proc_Code,Procedure_Description,Unit_Value,Basic_Rate,Child_Rate,ER_Rate,Conv_Ind,ER_Ind,Cutback_Ind,Prof_%,Rental_Rate,Non_Physn_Med_Prac_Ind
14749,1,A4640,REPLACEMENT PAD FOR USE WITH MEDIC,45.18,45.18,NaN,NaN,9,0,0,0.0,4.38,Y
14844,1,A7045,REPL EXHALATION PORT FOR PAP,16.44,16.44,NaN,NaN,9,0,0,0.0,1.64,Y
14965,1,B9002,ENTERAL NUTRITION INFUSION PUMP,907.45,907.45,NaN,NaN,9,0,0,0.0,87.89,Y
15075,1,E0100,CANES OF ALL MATRLSADJUST OR FIX W/TIP,15.99,15.99,NaN,NaN,9,0,0,0.0,4.75,Y
15076,1,E0105,"3 OR 4 PRNG CANE,ANY MTRLADJ/FIX W/TIP",37.50,37.50,NaN,NaN,9,0,0,0.0,7.09,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,1,V5140,"BINAURAL, BEHIND THE EAR",0.01,0.01,NaN,NaN,9,0,0,0.0,1.53,Y
18589,1,V5150,BINAURAL; GLASSES,0.01,0.01,NaN,NaN,9,0,0,0.0,1.53,Y
18593,1,V5190,"CROS HEARING AID, GLASSES",0.01,0.01,NaN,NaN,9,0,0,0.0,1.53,Y
18600,1,V5230,BICROS HEARING AID; GLASSES,0.01,0.01,NaN,NaN,9,0,0,0.0,1.53,Y


In [6]:
print(filtered_df_dict_2)

      Proc_Type Proc_Code                   Procedure_Description  Unit_Value  \
14749         1     A4640      REPLACEMENT PAD FOR USE WITH MEDIC       45.18   
14844         1     A7045            REPL EXHALATION PORT FOR PAP       16.44   
14965         1     B9002         ENTERAL NUTRITION INFUSION PUMP      907.45   
15075         1     E0100  CANES OF ALL MATRLSADJUST OR FIX W/TIP       15.99   
15076         1     E0105  3 OR 4 PRNG CANE,ANY MTRLADJ/FIX W/TIP       37.50   
...         ...       ...                                     ...         ...   
18588         1     V5140                BINAURAL, BEHIND THE EAR        0.01   
18589         1     V5150                       BINAURAL; GLASSES        0.01   
18593         1     V5190               CROS HEARING AID, GLASSES        0.01   
18600         1     V5230             BICROS HEARING AID; GLASSES        0.01   
18604         1     V5298                         HEARING AID NOC        0.01   

       Basic_Rate  Child_Ra

In [ ]:
#import pandas as pd

# Initialize an empty set to store unique SVC_IDs
unique_svc_ids = set()

# Iterate over each key (assuming each key is a dataframe) in filtered_df_dict_2
for key, df in final_df_2.items():
    # Extract SVC_ID column values and add them to the set
    unique_svc_ids.update(df['SVC_ID'].unique())

# Convert set of unique SVC_IDs to a DataFrame
final_Df_2 = pd.DataFrame({'SVC_ID': list(unique_svc_ids)})

# Print final_Df_2 to verify
print(final_Df_2)
